----
# PDF 논문 분석하기

## Task
1. 논문 검색 후 PDF 다운로드
2. PDF문서 embedding 후 VectorDB에 저장
3. 저장된 문서 기반으로 RetrievalQA 수행
4. Gradio로 Deploy

In [ ]:
# install pakage
!pip install langchain
!pip install arxiv
!pip install pypdf
!pip install openai
!pip install tiktoken
!pip install faiss-cpu
!pip install gradio

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 10.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.8/40.8 kB 4.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.4/49.4 kB 6.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 81.1/81.1 kB 2.2 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for sgmllib3k: filename=sgmllib3k-1.0.0-py3-none-any.whl size=6047 sha256=61671b3d8ec320c164bda7e11d664045d42c6f92df6b21a7466881d25ea0763b
  Stored in directory: /root/.cache/pip/wheels/f0/69/93/a47e9d621be168e9e33c7ce60524393c0b92ae83cf6c6e89c5
Successfully built sgmllib3k
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 276.6/276.6 kB 3.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.0/77.0 kB 2.0 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
llmx 0.0.15a

In [ ]:
import arxiv
from langchain.document_loaders import ArxivLoader
from langchain.document_loaders import PyPDFLoader
from langchain.chains import RetrievalQA
from langchain.llms import OpenAI
from langchain.llms import OpenAIChat
from langchain.text_splitter import RecursiveCharacterTextSplitter
import os
from langchain.vectorstores import FAISS
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.chat_models import ChatOpenAI

In [ ]:
os.environ["OPENAI_API_KEY"] = '/* Your KEY*/'

In [ ]:
# Load a paper to use
paper = next(arxiv.Search(query="chain of thought llms").results())
paper.download_pdf(filename="downloaded-paper.pdf")

<ipython-input-4-da053c656f04>:2: DeprecationWarning: The '(Search).results' method is deprecated, use 'Client.results' instead
  paper = next(arxiv.Search(query="chain of thought llms").results())


'./downloaded-paper.pdf'

In [ ]:
pdf_file_path = "downloaded-paper.pdf"

loader = PyPDFLoader(pdf_file_path)
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=2500,
    chunk_overlap=150,
    separators=["\n\n", "\n", "(?<=\. )", " ", ""]
)

paper_pages = loader.load_and_split(text_splitter)

In [ ]:
paper_pages[-1]

Document(page_content=',→p r o c e e d i n g t o buy t i c k e t s , t h a t I can r e f u n d t h e t i c k e t i f i t i s\n,→r e f u n d a b l e i s ” l e s s l i k e l y ” b e c a u s e I have a l r e a d y p u r c h a s e d t h e\n,→t i c k e t .\nFor s t e p 4 , a f t e r g e t t i n g an e m a i l c o n f i r m a t i o n of r e s e r v a t i o n , t h a t I\n,→can r e f u n d t h e t i c k e t i f i t i s r e f u n d a b l e i s ” more l i k e l y ” b e c a u s e\n,→I can check t h e t e r m s and c o n d i t i o n s of t h e t i c k e t and s e e i f i t\n,→i s r e f u n d a b l e or n o t .\nT h e r e f o r e , i n t h e f i n a l s t e p , t h e e v e n t i s ” more l i k e l y ” .\nAnswer : more l i k e l y\n. . .\nTable 34: CREPE with hints task demonstrations\n33', metadata={'source': 'downloaded-paper.pdf', 'page': 32})

In [ ]:
embeddings = OpenAIEmbeddings()
pages = paper_pages

db = FAISS.from_documents(pages, embeddings)

In [ ]:
gpt3_16 = ChatOpenAI(model_name = "gpt-3.5-turbo-16k", temperature=0.0)
gpt3 = ChatOpenAI(model_name = "gpt-3.5-turbo", temperature=0.0)
llm = gpt3_16

qa = RetrievalQA.from_chain_type(llm=llm, chain_type="stuff", retriever=db.as_retriever())

query = "논문의 내용을 요약해줘"

qa({"query": query})

{'query': '논문의 내용을 요약해줘',
 'result': '이 논문은 비용 효율적인 언어 모델 (LLM)을 사용하여 추론 작업을 수행하기 위한 간단하고 효과적인 파이프라인을 제안합니다. 이 파이프라인은 약한 LLM의 답변 일관성을 확인하여 LLM을 동적으로 결정하는 방식을 사용하여 비용을 절감합니다. 실험 결과, 이 파이프라인은 비용을 대폭 줄이면서도 비슷한 작업 성능을 달성할 수 있음을 보여줍니다. 또한, 중간 단계의 다양한 표현을 도입하여 약한 LLM에서 다양한 답변을 얻는 것이 최상의 성능을 보여줍니다. 이 연구는 LLM의 추론 능력을 저렴하게 활용함으로써 자원이 제한된 조직 및 기관에 혜택을 제공하고, 경제적인 절약과 환경 지속 가능성에도 기여할 수 있음을 강조합니다.'}

In [ ]:
import gradio as gr  # 그라디오 라이브러리를 불러옵니다.

def chatbot_response(user_message):

    # Use the provided RetrievalQA for getting the response
    qa = RetrievalQA.from_chain_type(llm=llm, chain_type="stuff", retriever=db.as_retriever())
    result = qa({"query": user_message})

    # Returning only result for simplicity in the Gradio interface
    return result.get('result')

# Create Gradio interface
interface = gr.Interface(
    fn=chatbot_response,
    inputs=gr.Textbox(placeholder="Type your message here..."),
    outputs="text",
    live=False
)

interface.launch()

Setting queue=True in a Colab notebook requires sharing enabled. Setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
Running on public URL: https://1bc6c0aa86f338153d.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from Terminal to deploy to Spaces (https://huggingface.co/spaces)
